In [ ]:
!pip install kaggle
!mkdir .kaggle

In [ ]:
import json
token = {"username":"fatemehy","key":"e6177841494493a99d3da8b61bab2fb1"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets list

- path is now set to: {/content}
ref                                                             title                                                size  lastUpdated          downloadCount  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge               COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-06-10 00:09:23           2998  
roche-data-science-coalition/uncover                            UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53           3209  
kwullum/fatal-police-shootings-in-the-us                        Fatal Police Shootings in the US                      1MB  2017-09-22 19:18:21          15320  
Cornell-University/arxiv                                        ArXiv Metadata                                        2GB  2020-05-06 23:18:35             71  
yash612

In [ ]:
!kaggle datasets list -s books
!kaggle datasets download -d jealousleopard/goodreadsbooks     -p /content
!unzip \*.zip

ref                                                        title                                               size  lastUpdated          downloadCount  
---------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
jealousleopard/goodreadsbooks                              Goodreads-books                                    622KB  2020-03-09 09:18:31           2677  
dareenalharthi/jamalon-arabic-books-dataset                Jamalon Arabic Books Dataset                         1MB  2019-08-15 18:58:06           1143  
zygmunt/goodbooks-10k                                      goodbooks-10k                                       12MB  2017-09-02 15:34:30          13833  
meetnaren/goodreads-best-books                             Goodreads' Best Books Ever                           2GB  2018-12-28 23:38:31           1186  
cwiloc/climate-data-from-ocean-ships                       Ocean Ship Logboo

In [ ]:
# Load data into Pandas Dataframe
import pandas as pd
df = pd.read_csv('books.csv',error_bad_lines=False)
df

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [ ]:
# Create a function named Popularity Based Recommender and use it to recommend books
# based on popularity.
# Use a weighted rank similar to that used in the IMDB rating example in Lesson 2. 
df.head(2).transpose()
#highest rating
recommendations = df.sort_values(by = 'average_rating',ascending =False).head(5)
print('recommendations: ',recommendations)
#maximum vote-count
recommendations = df.sort_values(by = 'ratings_count',ascending =False).head(5)
print('recommendations: ',recommendations)
#IMDB formula
# IMDB Weighted Rating (WR) = ((v/(v+m))*R)+((m/(v+m))*C)
# v is the number of votes for the movie
# m is the minimum votes to qualify
# R is the average rating of the movie
# C is the mean rating for all movies in the dataset
#Define the minimum vote count
minimum_vote_count = 0.75* df['ratings_count'].max()
#Define C – the mean rating
mean_rating = df['average_rating'].mean()
df['weighted_rating'] = (((df['ratings_count']/(df['ratings_count']+minimum_vote_count)) *
df['average_rating']) +
((minimum_vote_count/(df['ratings_count']+minimum_vote_count))*mean_rating))
recommendations = df.sort_values(by = 'weighted_rating',ascending = False).head(5)
print('recommendations: ',recommendations)

recommendations:        bookID  ...                     publisher
624     2034  ...  Oxford University Press  USA
9893   39829  ...                     Multnomah
4788   17224  ...         Square One Publishers
9324   36853  ...                  Disney Press
9720   38804  ...               Academica Press

[5 rows x 12 columns]
recommendations:         bookID  ...                  publisher
10336   41865  ...  Little  Brown and Company
1697     5907  ...           Houghton Mifflin
1462     5107  ...             Back Bay Books
307       960  ...               Pocket Books
3           5  ...            Scholastic Inc.

[5 rows x 12 columns]
recommendations:        bookID  ... weighted_rating
3          5  ...        4.187089
0          1  ...        4.174464
1          2  ...        4.147771
4415   15881  ...        4.128198
23        34  ...        4.096661

[5 rows x 13 columns]


In [ ]:
def Popularity_Based_Recommender():
  #IMDB formula
  # IMDB Weighted Rating (WR) = ((v/(v+m))*R)+((m/(v+m))*C)
  # v is the number of votes for the movie
  # m is the minimum votes to qualify
  # R is the average rating of the movie
  # C is the mean rating for all movies in the dataset
  #Define the minimum vote count
  minimum_vote_count = 0.75* df['ratings_count'].max()
  #Define C – the mean rating
  mean_rating = df['average_rating'].mean()
  df['weighted_rating'] = (((df['ratings_count']/(df['ratings_count']+minimum_vote_count)) *
  df['average_rating']) +
  ((minimum_vote_count/(df['ratings_count']+minimum_vote_count))*mean_rating))
  recommendations = df.sort_values(by = 'weighted_rating',ascending = False).head(5)
  print('recommendations: ',recommendations)
  return recommendations

Popularity_Based_Recommender()


recommendations:        bookID  ... weighted_rating
3          5  ...        4.187089
0          1  ...        4.174464
1          2  ...        4.147771
4415   15881  ...        4.128198
23        34  ...        4.096661

[5 rows x 13 columns]


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,weighted_rating
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,4.187089
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,4.174464
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,4.147771
4415,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.,4.128198
23,34,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.36,0618346252,9780618346257,eng,398,2128944,13670,9/5/2003,Houghton Mifflin Harcourt,4.096661


In [ ]:
# Content Based Recommender
# Create a function named Content Based Recommender and use it to recommend books
# based on content.
# Step 4-a
# TF-IDF Vectorizer
# Use TF-IDF Vectorizer on the author data for each book.
# Step 4-b
# Distance matrix
# Choose cosine similarity for pairwise distances comparison.
def ContentBasedRecommender(title, indices, distance_matrix):
 id_ = indices[title]
 print('id',id_)
 distances = list(enumerate(distance_matrix[id_]))
 distances = sorted(distances, key=lambda x: x[0], reverse = True)
 #Finding the top 15 authors with the most number of books
#  df['authors'].value_counts().head(10) 
 distances = distances[1:6]
 recommendations = [distance[0] for distance in distances]
 return df['authors'].iloc[recommendations]


def main():
#  df = pd.read_csv('books.csv',error_bad_lines=False)
 from sklearn.feature_extraction.text import TfidfVectorizer
 cbr = TfidfVectorizer(stop_words='english')
 df[['authors']] = df[['authors']].fillna('')
 tfidf_matrix = cbr.fit_transform(df['authors'])

 from sklearn.metrics.pairwise  import cosine_similarity

 distance_matrix = cosine_similarity(tfidf_matrix)
 indices = pd.Series(df.index, index=df['authors']).drop_duplicates()
 print('indices',indices)

 return ContentBasedRecommender('William T. Vollmann', indices,distance_matrix)
main()

indices authors
J.K. Rowling/Mary GrandPré                                    0
J.K. Rowling/Mary GrandPré                                    1
J.K. Rowling                                                  2
J.K. Rowling/Mary GrandPré                                    3
J.K. Rowling/Mary GrandPré                                    4
                                                          ...  
William T. Vollmann/Larry McCaffery/Michael Hemmingson    11118
William T. Vollmann                                       11119
William T. Vollmann                                       11120
William T. Vollmann                                       11121
Mark Twain                                                11122
Length: 11123, dtype: int64
id William T. Vollmann     3393
William T. Vollmann    11117
William T. Vollmann    11119
William T. Vollmann    11120
William T. Vollmann    11121
dtype: int64


3    J.K. Rowling/Mary GrandPré
2                  J.K. Rowling
1    J.K. Rowling/Mary GrandPré
0    J.K. Rowling/Mary GrandPré
Name: authors, dtype: object